In [1]:
# loand Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import sys
import os
from windrose import WindroseAxes

# for a folder that contain scripts for modularity 
sys.path.append(os.path.abspath('../scripts'))
sys.path.append(os.path.abspath('../src'))

from data_loader import get_file_path, get_cleaned_data, CSVData

In [5]:
# Load Benin data
togo_path = get_file_path("togo")
togo_data = CSVData(togo_path).load_data()

In [6]:
togo_data.head()

,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-10-25 00:01,-1.3,0.0,0.0,0.0,0.0,24.8,94.5,0.9,1.1,0.4,227.6,1.1,977,0,0.0,24.7,24.4,NaN
1,2021-10-25 00:02,-1.3,0.0,0.0,0.0,0.0,24.8,94.4,1.1,1.6,0.4,229.3,0.7,977,0,0.0,24.7,24.4,NaN
2,2021-10-25 00:03,-1.3,0.0,0.0,0.0,0.0,24.8,94.4,1.2,1.4,0.3,228.5,2.9,977,0,0.0,24.7,24.4,NaN
3,2021-10-25 00:04,-1.2,0.0,0.0,0.0,0.0,24.8,94.3,1.2,1.6,0.3,229.1,4.6,977,0,0.0,24.7,24.4,NaN
4,2021-10-25 00:05,-1.2,0.0,0.0,0.0,0.0,24.8,94.0,1.3,1.6,0.4,227.5,1.6,977,0,0.0,24.7,24.4,NaN


In [9]:
# Copying the dataset
df3 = togo_data.copy()

In [10]:
# Missing values for each column
missing_values = df3.isna().sum()
missing_values

Timestamp             0
GHI                   0
DNI                   0
DHI                   0
ModA                  0
ModB                  0
Tamb                  0
RH                    0
WS                    0
WSgust                0
WSstdev               0
WD                    0
WDstdev               0
BP                    0
Cleaning              0
Precipitation         0
TModA                 0
TModB                 0
Comments         525600
dtype: int64

In [11]:
# Summary statistics for all numeric columns
summary_stats = df3.describe()
summary_stats

,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
count,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,0.0
mean,230.555040,151.258469,116.444352,226.144375,219.568588,27.751788,55.013160,2.368093,3.229490,0.557740,161.741845,10.559568,975.915242,0.000535,0.001382,32.444403,33.543330,NaN
std,322.532347,250.956962,156.520714,317.346938,307.932510,4.758023,28.778732,1.462668,1.882565,0.268923,91.877217,5.915490,2.153977,0.023116,0.026350,10.998334,12.769277,NaN
min,-12.700000,0.000000,0.000000,0.000000,0.000000,14.900000,3.300000,0.000000,0.000000,0.000000,0.000000,0.000000,968.000000,0.000000,0.000000,13.100000,13.100000,NaN
25%,-2.200000,0.000000,0.000000,0.000000,0.000000,24.200000,26.500000,1.400000,1.900000,0.400000,74.800000,6.900000,975.000000,0.000000,0.000000,23.900000,23.600000,NaN
50%,2.100000,0.000000,2.500000,4.400000,4.300000,27.200000,59.300000,2.200000,2.900000,0.500000,199.100000,10.800000,976.000000,0.000000,0.000000,28.400000,28.400000,NaN
75%,442.400000,246.400000,215.700000,422.525000,411.000000,31.100000,80.800000,3.200000,4.400000,0.700000,233.500000,14.100000,977.000000,0.000000,0.000000,40.600000,43.000000,NaN
max,1424.000000,1004.500000,805.700000,1380.000000,1367.000000,41.400000,99.800000,16.100000,23.100000,4.700000,360.000000,86.900000,983.000000,1.000000,2.300000,70.400000,94.600000,NaN


In [12]:
# Percentage of missing values per column
missing_percentage = (missing_values / len(df3)) * 100

In [13]:
# Columns with more than 5% missing values
columns_above_5pt_missing = missing_percentage[missing_percentage > 5].index.tolist()

In [14]:
print("\n🚨 Columns with >5% Missing Values:")
for col in columns_above_5pt_missing:
    print(f"{col}: {missing_percentage[col]:.2f}% missing")


🚨 Columns with >5% Missing Values:
Comments: 100.00% missing


In [20]:
df3 = df3.drop(columns=['Comments'], axis=1)

In [21]:
# Identify target columns
target_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Check for missing 
missing_in_terget = df3[target_cols].isna().sum()
missing_in_terget

GHI       0
DNI       0
DHI       0
ModA      0
ModB      0
WS        0
WSgust    0
dtype: int64

In [22]:
# 2. Compute Z-scores and flag outliers (|Z| > 3)
z_scores = df3[target_cols].apply(lambda x: zscore(x, nan_policy='omit'))

# Create a boolean mask for rows where any column has |Z| > 3
outlier_mask = (z_scores.abs() > 3)

In [23]:
# Count of outliers per column
outliers = outlier_mask.sum()
outliers

GHI        305
DNI       1062
DHI       3415
ModA       137
ModB       206
WS        3510
WSgust    3915
dtype: int64

In [24]:
# Flag rows with any outliers
df3['has_outlier'] = outlier_mask.any(axis=1)

# Impute missing values in target columns with the median
df3[target_cols] = df3[target_cols].apply(lambda col: col.fillna(col.median()))

# Drop rows that still have missing values (if any)
df_cleaned = df3.dropna(subset=target_cols)

In [25]:
# Export cleaned data
country = "togo"  
output_path = f"data/{country}_clean.csv"
os.makedirs("data", exist_ok=True)
df_cleaned.to_csv(output_path, index=False)

print(f"✅ Cleaned data saved to: {output_path}")

✅ Cleaned data saved to: data/togo_clean.csv
